In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
%matplotlib qt

In [68]:
def prepare():
    '''
    data preparation;
    '''
    df = pd.read_csv('panl.csv')
    df['entry_time']=pd.to_datetime(df['order_entry_time'])
    df['cancel_time']=pd.to_datetime(df['cancel_entry_time'])
    df['exec_time']=pd.to_datetime(df['execution_time'])
    df=df.sort_values(by=['entry_time','orderid','order_type'],ascending=[True,True,False])
    df=df[['orderid','side','order_type','prc','q_new','q_exec','q_canc','prc_exec','entry_time','cancel_time','exec_time']]
    return df

In [69]:
def plot_oid(oid):
    '''
    plot single order.
    marker size of the plot can be changed by changing ms value;
    oid: order number
    '''
    s=2.0
    ms=10 #change marker size here;
    df=global_df[global_df['orderid']==oid]
    
    if df.iloc[0]['side']=='B':
        _color='blue' #blue color for buy side;
    else:
        _color='red'  #red color for sell side;
        
    start_price=None
    start_time=None
    exec_price=[]
    exec_time=[]
    cancel_price=None
    cancel_time=None
    price_list=[]
    time_list=[]
    
    start_time=df.iloc[0]['entry_time']
    start_price=df.iloc[0]['prc']
    q=df.iloc[0]['q_new']
    
    plt.plot(start_time,start_price,marker='s',markersize=np.log(q)*s,color=_color,fillstyle='none')
    price_list.append(start_price)
    time_list.append(start_time)
    
    qq=0.0
    for i in range(1,len(df)):
        if df.iloc[i]['order_type']=='EXECUTION':
            price=df.iloc[i]['prc_exec']
            time=df.iloc[i]['exec_time']
            #plt.plot(time,price,marker='v',color=_color,fillstyle='none')
            #price_list.append(price)
            #time_list.append(time)
            
            exec_price.append(price)
            exec_time.append(time)
            qq+=df.iloc[i]['q_exec']
            
        elif df.iloc[i]['order_type']=='CANCEL':
            cancel_price=df.iloc[0]['prc']
            cancel_time=df.iloc[0]['cancel_time']
            qqq=df.iloc[0]['q_canc']
            plt.plot(cancel_time,cancel_price,marker='x',markersize=np.log(qqq)*s,color=_color,fillstyle='none')
            #price_list.append(price)
            #time_list.append(time)
    qq=np.log(qq)*s
    if len(exec_price)==1:
        plt.plot(exec_time[0],exec_price[0],marker='v',markersize=qq,color=_color,fillstyle='none')
        time_list.append(exec_time[0])
        price_list.append(exec_price[0])
        
    elif len(exec_price)>=2:
        plt.plot(exec_time[0],exec_price[0],marker='v',markersize=qq,color=_color,fillstyle='none')
        plt.plot(exec_time[-1],exec_price[-1],marker='v',markersize=qq,color=_color,fillstyle='none')
        time_list.append(exec_time[0])
        price_list.append(exec_price[0])
        time_list.append(exec_time[-1])
        price_list.append(exec_price[-1])
        
    time_list.append(cancel_time)
    price_list.append(cancel_price)
        
    plt.plot(time_list,price_list,color=_color)
    
    if len(exec_price)==0:
        return (None,None)
    if len(exec_price)==1:
        return (exec_time[0],exec_price[0])
    if len(exec_price)>=2:
        return (exec_time[0]+(exec_time[-1]-exec_time[0])/2,np.average(exec_price))

In [70]:
def show_graph(oid=49244374,num=50,dy=0.1,dx=10):
    '''
    main function, plot all orders;
    oid: order number
    num: order range 
    dy: y axis range
    dx: x axis range(in seconds)
    '''
    
    oid_list=global_df['orderid'].unique()
    idx=np.nonzero(oid_list==oid)[0][0]
    
    time_list=[]
    price_list=[]
    
    time,price=plot_oid(oid)
    if price is not None:
        time_list.append(time)
        price_list.append(price)
    
    for i in range(1,num):
        
        time,price=plot_oid(oid_list[idx-i])
        if price is not None:
            time_list.append(time)
            price_list.append(price)
        
        time,price=plot_oid(oid_list[idx+i])
        if price is not None:
            time_list.append(time)
            price_list.append(price)
            
    tuple_list=zip(time_list,price_list)
    tuple_list=sorted(tuple_list)
    time_list,price_list=zip(*tuple_list)
    
            
    #import pdb;pdb.set_trace()
    plt.plot(time_list,price_list,color='green')
    plt.xlim(min(time_list)-timedelta(seconds=dx),max(time_list)+timedelta(seconds=dx))
    plt.ylim(min(price_list)-dy,max(price_list)+dy)
    plt.xlabel('Time')
    plt.ylabel('Price')
    
    _title='order id:'+str(oid)+'  range: '+str(num)+'  (blue:buy red:sell)'
    plt.title(_title)
    plt.grid()
    
    plt.show()
    

In [77]:
global_df=prepare()
show_graph(oid=49244374,num=65,dx=25,dy=0.05) 